# Project 2: How to Keep your Burrito Warm

### First Draft

Alex Wenstrup and Robin Graham-Hayes

## Question

Suppose you buy a chipotle burrito (or any other burrito) for takeout, and want it to be as warm as possible when you get home. If you lead a lonely life of luxury and have a vacant but heated passenger seat in your car, you may be inclined to turn on your heated seat and strap your burrito in for a toasty ride. If, however, you don't find yourself in that position, are you better off leaving your burrito in the cooler, but insulated seat next to you, or on your lap, which starts off warmer, but will be cooled by your bloodstream drawing the heat throughout your body?

In [6]:
# Configure Jupyter so figures appear in the notebook
%matplotlib inline

# Configure Jupyter to display the assigned value after an assignment
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'

# import functions from the modsim.py module
from modsim import *

In [7]:
tBurrito=24 #above room temp
tLap=34     #Source: https://hypertextbook.com/facts/2001/AbantyFarzana.shtml
tCushion=22 # C room temp

state = State(tBurrito=tburrito, #create state
              tLap=tLap,
              tCushion=tCushion,)

NameError: name 'tburrito' is not defined

In [ ]:
#https://www.sciencedirect.com/science/article/pii/S026087740600392X
